# Colaboratory setup

In [1]:
# hdf5
# pytorch lightning
# wandb
# hydra
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
import subprocess
import warnings
warnings.filterwarnings("ignore")

CUDA_version = [s for s in subprocess.check_output(["nvcc", "--version"]).decode("UTF-8").split(", ") if s.startswith("release")][0].split(" ")[-1]
print("CUDA version:", CUDA_version)
if CUDA_version == "10.0":
    torch_version_suffix = "+cu100"
elif CUDA_version == "10.1":
    torch_version_suffix = "+cu101"
elif CUDA_version == "10.2":
    torch_version_suffix = ""
else:
    torch_version_suffix = "+cu110"

!pip install torch==1.7.1{torch_version_suffix} torchvision==0.8.2{torch_version_suffix} -f https://download.pytorch.org/whl/torch_stable.html ftfy regex
!pip install ftfy==5.8
!pip install transformers
!pip install open_clip_torch
!pip install pytorch-lightning
!pip install pytorch-memlab

import matplotlib.pyplot as plt
import numpy as np
from PIL import Image, UnidentifiedImageError
import open_clip
import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import transformers
import torch
import pytorch_lightning as pl

%cd "/content/drive/MyDrive/Colab Notebooks/mclip-finetune"

CUDA version: 11.2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
tcmalloc: large alloc 1156751360 bytes == 0x34d8000 @  0x7f5ed0df71e7 0x4b2590 0x5ad01c 0x5dcfef 0x58f92b 0x590c33 0x5e48ac 0x4d20fa 0x51041f 0x58fd37 0x50ca37 0x58fd37 0x50ca37 0x5b575e 0x4bad0a 0x4d3249 0x591e56 0x50e18c 0x5b4ee6 0x4bad0a 0x4d3249 0x591e56 0x50e18c 0x5b4ee6 0x4bad0a 0x4d3249 0x591e56 0x50e18c 0x5b575e 0x4bad0a 0x4d3249
tcmalloc: large alloc 1445945344 bytes == 0x48402000 @  0x7f5ed0df8615 0x58ead6 0x4f355e 0x4d222f 0x51041f 0x5b4ee6 0x58ff2e 0x50ca37 0x5b4ee6 0x58ff2e 0x50ca37 0x5b4ee6 0x58ff2e 0x50ca37 0x5b4ee6 0x58ff2e 0x50ca37 0x5b4ee6 0x58ff2e 0x50ca37 0x58fd37 0x50ca37 0x5b4ee6 0x58ff2e 0x50d482 0x5b4ee6 0x58ff2e 0x50ca37 0x5b575e 0x58ff2e 0x50ca37
  Using cached https://download.pytorch.org/whl/cu110/torch-1.7.1%2Bcu110-cp37-cp37m-linux_x86_64.whl (1156.8 MB)
  Using cached htt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached torch-1.13.0-cp37-cp37m-manylinux1_x86_64.whl (890.2 MB)
  Using cached torchvision-0.14.0-cp37-cp37m-manylinux1_x86_64.whl (24.3 MB)
  Attempting uninstall: torch
    Found existing installation: torch 1.7.1+cu110
    Uninstalling torch-1.7.1+cu110:
      Successfully uninstalled torch-1.7.1+cu110
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.8.2+cu110
    Uninstalling torchvision-0.8.2+cu110:
      Successfully uninstalled torchvision-0.8.2+cu110
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.13.1 requir

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 93 kB 1.2 MB/s 
  Created wheel for pytorch-memlab: filename=pytorch_memlab-0.2.4-py3-none-any.whl size=22857 sha256=3ad043e5b9f4ddac19825ad4a85a1dd3ca43ca7446ba0528128b1a8295e73506
  Stored in directory: /root/.cache/pip/wheels/bb/c4/8e/64b321e10754783fb334f883d21b651860ee0737ffdbf688c7
  Created wheel for calmsize: filename=calmsize-0.1.3-py3-none-any.whl size=2887 sha256=6474d5f9c7cfdfb6da10236fbb6549b0d806a8e7cebc8cd9d9ffa92139dab0e8
  Stored in directory: /root/.cache/pip/wheels/a3/92/5f/c4c4e3fa257084aecbdc44c0fe60bdef0cb870603f3642dd96
Successfully built pytorch-memlab calmsize
/content/drive/MyDrive/Colab Notebooks/mclip-finetune


# Finetuning

In [6]:
import importlib

import coco2014
import mclip
importlib.reload(coco2014)
importlib.reload(mclip)

data = coco2014.DataModule(32, 16)
model = mclip.MClipModelModule()
#trainer = pl.Trainer(max_epochs=20, accelerator='gpu', devices=1)
trainer = pl.Trainer(max_epochs=20)
trainer.fit(model, data)

Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaModel: ['lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:

Training: 0it [00:00, ?it/s]

In [7]:
!free

              total        used        free      shared  buff/cache   available
Mem:       26690640    14887732      459720        1284    11343188    16062572
Swap:             0           0           0
